In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt

In [ ]:
RUNS = { "noghc": "decr_14_3h_ws",
        "annual": "decr_16_3h_ws",
        "monthly": "decr_15_3h_ws",
        "hourly": "decr_13_3h_ws",
       }

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
#OUTPUT = PREFIX + "results/" + RUN + "/graphics/comp_benefits/"

In [ ]:
def get_data(stats, h2export, opts):
    """Filter stats according to h2export value and opts value
    """
    cost = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"cost"]
    mg_AC_exclu_H2 = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"mg_AC_exclu_H2 El_all"]
    mg_H2_False_False_exportonly = stats[(stats.index == h2export) & (stats.opts == opts)].loc[:,"mg_H2_False_False_exportonly"]
    return cost, mg_AC_exclu_H2, mg_H2_False_False_exportonly 

In [ ]:
def prepare_data(ghrules, h2export, opts):
    df = pd.DataFrame(index=ghrules, columns=["cost", "mg_AC_exclu_H2", "mg_H2_False_False_exportonly"])

    for rule in ghrules:
        INPUT = PREFIX + "results/" + RUNS[rule] + "/integrated_comp/stats.csv"
        stats = pd.read_csv(INPUT, index_col=0)
        cost, mg_AC_exclu_H2, mg_H2_False_False_exportonly = get_data(stats, h2export, opts)    
        df.loc[rule, "cost"] = cost.values[0]
        df.loc[rule, "mg_AC_exclu_H2"] = mg_AC_exclu_H2.values[0]
        df.loc[rule, "mg_H2_False_False_exportonly"] = mg_H2_False_False_exportonly.values[0]    

    # Add relative values in % based on noghc row
    df["cost_rel"] = df["cost"] / df.loc["noghc", "cost"] * 100
    df["mg_AC_exclu_H2_rel"] = df["mg_AC_exclu_H2"] / df.loc["noghc", "mg_AC_exclu_H2"] * 100
    df["mg_H2_False_False_exportonly_rel"] = df["mg_H2_False_False_exportonly"] / df.loc["noghc", "mg_H2_False_False_exportonly"] * 100
    return df  

### Relative, subplot

In [ ]:
h2exports = [200, 1]
opts = ["Co2L2.0", "Co2L0.00"]
ghrules = ["noghc","annual","monthly", "hourly"]

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 6), sharex=True, sharey=True)

n=1
for h2export in h2exports:
    for o in opts:
        ax = plt.subplot(2, 2, n)
        n += 1

        df = prepare_data(ghrules, h2export, o)

        ax.plot(df.index, df["cost_rel"], label="Total costs", color="black")
        ax.plot(df.index, df["mg_H2_False_False_exportonly_rel"], label="H2 costs paid by exporters", color="orange")
        ax.plot(df.index, df["mg_AC_exclu_H2_rel"], label="Local electricity price", color="green")

        ax.set_title(f"{h2export} TWh export, {o} CO2")
        ax.grid(axis='y')
        ax.set_ylim(0, 135)

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc = (0.4,-0.01))
fig.text(0.04, 0.5, 'Change in %', va='center', rotation='vertical')

plt.show()

### Relative, single

In [ ]:
df = prepare_data(ghrules, 200, "Co2L2.0")
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(df.index, df["cost_rel"], label="Total costs", color="black")
ax.plot(df.index, df["mg_H2_False_False_exportonly_rel"], label="H2 costs paid by exporters", color="orange")
ax.plot(df.index, df["mg_AC_exclu_H2_rel"], label="Local electricity price", color="green")

ax.set_xlabel("H2 rule")
ax.set_ylabel("Change in % compared to no H2 rule")
ax.set_title(f"200 TWh export, 0% CO2 reduction")
ax.set_ylim(0, 135)
ax.grid(axis='y')

ax.legend(loc="lower left")

### Absolute values

In [ ]:
df = prepare_data(ghrules, 200, "Co2L2.0")
fig, ax = plt.subplots(figsize=(6, 4))

#ax.plot(df.index, df["cost"]/1e9, label="Total cost")
ax.plot(df.index, df["mg_AC_exclu_H2"], label="Local electricity price", color="green")
ax.plot(df.index, df["mg_H2_False_False_exportonly"], label="H2 costs paid by exporters", color="orange")


# Add secondary axis for total cost
ax2 = ax.twinx()
ax2.plot(df.index, df["cost"]/1e9, label="Total cost", color="black")
ax2.set_ylabel("Total cost in B€")
ax2.set_ylim(5, 30)

ax.set_xlabel("H2 rule")
ax.set_ylabel("Marginal price of electricity/h2 in €/MWh")
ax.set_ylim(0, 100)

ax.legend(loc="lower left")
ax2.legend(loc="lower right")

ax.grid(axis='y')

In [ ]:
#ax.legend()
if not os.path.isdir(OUTPUT):
    os.makedirs(OUTPUT)
    print("Created directory " + OUTPUT)
plt.savefig(OUTPUT + "electricity" + str(export) + ".pdf", bbox_inches="tight")
plt.show()